In [1]:
library(pracma)
#load source file
src.path <- "../src"
source(paste(src.path, "DataGenerationProcess", "synth_basis.R", sep="/"))
source(paste(src.path, "DataGenerationProcess", "synth_data.R", sep="/"))
source(paste(src.path, "DataGenerationProcess", "synth_graph.R", sep="/"))
source(paste(src.path, "DataGenerationProcess", "synth_linearop.R", sep="/"))

source(paste(src.path, "Estimation", "basis_estimation.R", sep="/"))
source(paste(src.path, "Utility", "utlity.R", sep="/"))

source(paste(src.path, "Estimation", "cca_estimation.R", sep="/"))
library(viridis)
library(fields)

 要求されたパッケージ splines をロード中です 

 要求されたパッケージ Matrix をロード中です 


 次のパッケージを付け加えます: ‘Matrix’ 


 以下のオブジェクトは ‘package:pracma’ からマスクされています: 

     expm, lu, tril, triu 


 要求されたパッケージ fds をロード中です 

 要求されたパッケージ rainbow をロード中です 

 要求されたパッケージ MASS をロード中です 

 要求されたパッケージ pcaPP をロード中です 

 要求されたパッケージ RCurl をロード中です 

 要求されたパッケージ deSolve をロード中です 


 次のパッケージを付け加えます: ‘deSolve’ 


 以下のオブジェクトは ‘package:pracma’ からマスクされています: 

     rk4 



 次のパッケージを付け加えます: ‘fda’ 


 以下のオブジェクトは ‘package:graphics’ からマスクされています: 

     matplot 


 要求されたパッケージ viridisLite をロード中です 

 要求されたパッケージ spam をロード中です 

 要求されたパッケージ dotCall64 をロード中です 

 要求されたパッケージ grid をロード中です 

Spam version 2.7-0 (2021-06-25) is loaded.
Type 'help( Spam)' or 'demo( spam)' for a short introduction 
and overview of this package.
Help for individual functions is also obtained by adding the
suffix '.spam' to the function name, e.g. 'help( chol.spam)'.


 次のパッケージを付け加えます: ‘spam’ 


 以下のオブジェクトは ‘package:mvtnorm’ からマスクされています: 

     rmvnorm 


 以下のオブジェクトは ‘package:Matr

In [2]:
n <- 100
p <- 50
k.gen <- 11
M <- 2
obs.time <- seq(0,1,1/50)
## be careful for the choice of the number of basis function 
## fourier basis: km must be odd
## bspline basis km>4
km.gen <- c(5,13)

### Canonical Correlation Analysis
Remember to set the data generation to be ``dependent``. If set ``indepdendent``, the resulting canonical correlation would go to zero as $n$ goes to inifinity, which matches the thoery. 

In [4]:
#remember to set the cca value to be dependent data, the results quite differ a lot


n <- 100 #vary this to see the outcomes
p <- 50
k <- 15
M <- 2
obs.time <- seq(0,1,1/50)
## be careful for the choice of the number of basis function 
## fourier basis: km must be odd
## bspline basis km>4
km1 <- c(15,15)

##data generation process 1##
# generate latent covariance matrix
cov <- solve(synth.omega.tridiag4(p,k))
# generate Amatrix, Noisematrix, basis matrix
Apinv_list <- list()
N_list <- list()
basis.m_list <- list()


for(m in 1:M){
    Am <- synth.linear_op.sparse_orthogonal(k,km1[m], floor(min(km1[m],k)/2), scale=1)
    #Am <- synth.linear_op.identity(k)
    
    Apinv_list[[m]] <- pinv(Am)
    N_list[[m]] <- diag(p*km1[m])*0.
    basis.m_list[[m]] <- synth.fourier.bases.m(obs.time, km1[m])
}

km.est <- km.gen
for(i in 1:length(km.est)){
    
    if(km.est[i]  > k.gen){
        km.est[i] <- k.gen
    }
}

k.est <- min(km.est)
par(bg="white", mfrow=c(2,2))
n_list <- c(20,100,500,5000)
for(n in n_list){
data <- synth.data_from_graph(n, p, cov, basis.m_list, Apinv_list, N_list, dependent=FALSE, addnoise=FALSE)
##compute mean of the data across p
data_mean <- utlity.mean_data(data) # n \times p
##compute functional basis of each modality
fbasis_list <- list()
for(m in 1:M){
    #print(paste("Esimtating funcational basis of data modality",m))
    fbasis_list[[m]] <- estimate.fpca.basis(c(0,1), obs.time, t(data_mean[[m]]), km.est[m], bs.nbasis=km.est[m]*3)$vectors
}
#data modality 1
d1 <-Data2fd(argvals=obs.time, y=t(data_mean[[1]]), basisobj=fbasis_list[[1]])
#data modality 2
d2 <-Data2fd(argvals=obs.time, y=t(data_mean[[2]]), basisobj=fbasis_list[[2]])
k_est <- min(km1[1],km1[2])
ccoar <- cca.fd(d1, d2, ncan = k_est)$ccacorr

plot(ccoar,xlab="canonical variable", ylab="canonical correlation",main=paste("CCA for independent data, n = ",n))
}

ERROR: Error in chol.default(Sigma, ...):  次数 1 の主対角行列が正定値ではありません 


In [ ]:
#remember to set the cca value to be dependent data, the results quite differ a lot


n <- 1000 #vary this to see the outcomes
p <- 50
k.gen <-15
M <- 2
obs.time <- seq(0,1,1/50)
## be careful for the choice of the number of basis function 
## fourier basis: km must be odd
## bspline basis km>4
km.gen <- c(15, 15)
##data generation process 1##
# generate latent covariance matrix
cov <- solve(synth.omega.tridiag4(p, k.gen))
# generate Amatrix, Noisematrix, basis matrix
Apinv_list <- list()
A_list <- list()
N_list <- list()
basis.m_list <- list()
true.basis_list <- list()
true.values_list <- list()


for(m in 1:M){
    Am <- synth.linear_op.sparse_orthogonal(k.gen, km.gen[m], min(km.gen[m],k.gen), scale=1)
    #Am <- synth.linear_op.identity(k.gen)
    #Am <- synth.linear_op.orthogonal(k.gen,km.gen[m])
    #for(j in 1:k.gen){
    #    Am[j,] <-  Am[j,] / (2*j)
    #}
    A_list[[m]] <- Am
    Apinv_list[[m]] <- pinv(Am)

    N_list[[m]] <- diag(p*km.gen[m])*0
    basis.m_list[[m]] <- synth.fourier.bases.m(obs.time, km.gen[m])

    eigen.r <- synth.true.basis(obs.time, cov, pinv(Am), basis.m_list[[m]])
    #print(eigen.r$vectors)
    true.basis_list[[m]] <- eigen.r$vectors
    true.values_list[[m]] <- eigen.r$values #make sure that the eigenvalue must be distinct to get unique eigenbasis
    print(paste("true eigenvalues of modality",m,true.values_list[[m]]))
}



In [ ]:
n_list <- c(1000)


for(n in n_list){
    data <- synth.data_from_graph(n, p, cov, basis.m_list, Apinv_list, N_list, dependent=TRUE, addnoise=FALSE)
    ##compute mean of the data across p
    data_mean <- utlity.mean_data(data) # n \times p
    ##compute functional basis of each modality
    par(bg="white", mfrow=c(2,2))
    fbasis_list <- list()
    for(m in 1:M){
        mean_cov <- t(data_mean[[m]]) %*% data_mean[[m]] / n
        image(mean_cov)
        #print(paste("Esimtating funcational basis of data modality",m))
        fbasis_list[[m]] <- estimate.fpca.basis(c(0,1), obs.time, t(data_mean[[m]]), km.gen[m], bs.nbasis=km.est[m]*3)$vectors
    }
    #data modality 1
    d1 <-Data2fd(argvals=obs.time, y=t(data_mean[[1]]), basisobj=fbasis_list[[1]])
    #data modality 2
    d2 <-Data2fd(argvals=obs.time, y=t(data_mean[[2]]), basisobj=fbasis_list[[2]])

    ccoar <- cca.fd(d1, d2)$ccacorr
    #print(ccoar)
    par(bg="white", mfrow=c(2,3))
    for(m in 1:M){
        plot(true.basis_list[[m]])
        plot( fbasis_list[[m]])
        image(inprod(true.basis_list[[m]],  fbasis_list[[m]]))
    }
}



In [ ]:
data <- synth.data_from_graph(100, p, cov, basis.m_list, Apinv_list, N_list, dependent=TRUE, addnoise=FALSE)
##compute mean of the data across p
data_mean <- utlity.mean_data(data)
for(m in 1:M){
    #adjust number of basis to estimate
    temp_values <- estimate.fpca.basis(c(0,1), obs.time, t(data_mean[[m]]), km.est[m], bs.nbasis=km.est[m]*3)$values
    #print(paste("Number of km before adjustment", km.est[m]))
    km.est[m]<-length(which(temp_values>1e-8))
    #print(paste("Number of km after adjustment", km.est[m]))

    fbasis_list[[m]] <- estimate.fpca.basis(c(0,1), obs.time, t(data_mean[[m]]), km.est[m], bs.nbasis=km.est[m]*3)$vectors

    fvalue_list[[m]] <- estimate.fpca.basis(c(0,1), obs.time, t(data_mean[[m]]), km.est[m], bs.nbasis=km.est[m]*3)$values[1:km.est[m]]
    print(fvalue_list[[m]])
}

d1 <-Data2fd(argvals=obs.time, y=t(data_mean[[1]]), basisobj=fbasis_list[[1]])
#data modality 2

d2 <-Data2fd(argvals=obs.time, y=t(data_mean[[2]]), basisobj=fbasis_list[[2]])
k.est <- min(km.est)


par(bg="white", mfrow=c(2,2))


cca.r.est <- estimate.cca(d1,d2,  fbasis_list[[1]],  fbasis_list[[2]], fvalue_list[[1]], fvalue_list[[2]], ncan=k.est)
print(cca.r.est$corr)

image.plot(cca.r.est$A1, col=viridis(64), xlim=c(-.1, 1.1), ylim=c(1.1, -0.1), main="estimated A1")
image.plot(pinv(Apinv_list[[1]]), col=viridis(64), xlim=c(-.1, 1.1), ylim=c(1.1, -0.1),  main="true A1")

#image.plot(cca.r.true$A2, col=viridis(64))
image.plot(cca.r.est$A2, col=viridis(64), xlim=c(-.1, 1.1), ylim=c(1.1, -0.1), main="estimated A2")
image.plot(pinv(Apinv_list[[2]]), col=viridis(64), xlim=c(-.1, 1.1), ylim=c(1.1, -0.1), main="true A2")


In [ ]:
N.level = c(0.1,0.5,0.7,1.0,1.5,2.0)
fbasis_list <- list()
fvalue_list <- list()
run_simulation <- function(noise){

    for(m in 1:M){
        idx <- (1- seq(0,1,1 / (p*km.gen[m]-1)) ** 3)
        
        N_list[[m]] <-  diag(idx)*noise
    } 
    data <- synth.data_from_graph(n, p, cov, basis.m_list, Apinv_list, N_list, dependent=TRUE, addnoise=FALSE)
    ##compute mean of the data across p
    data_mean <- utlity.mean_data(data) # n \times p
    for(m in 1:M){
        #adjust number of basis to estimate
        temp_values <- estimate.fpca.basis(c(0,1), obs.time, t(data_mean[[m]]), km.est[m], bs.nbasis=km.est[m]*3)$values
        #print(paste("Number of km before adjustment", km.est[m]))
        km.est[m]<-length(which(temp_values>1e-8))
        #print(paste("Number of km after adjustment", km.est[m]))

        fbasis_list[[m]] <- estimate.fpca.basis(c(0,1), obs.time, t(data_mean[[m]]), km.est[m], bs.nbasis=km.est[m]*3)$vectors

        fvalue_list[[m]] <- estimate.fpca.basis(c(0,1), obs.time, t(data_mean[[m]]), km.est[m], bs.nbasis=km.est[m]*3)$values[1:km.est[m]]
        print(fvalue_list[[m]])
    }
    print(paste("Noise level:", noise))
    d1 <-Data2fd(argvals=obs.time, y=t(data_mean[[1]]), basisobj=fbasis_list[[1]])
    #data modality 2

    d2 <-Data2fd(argvals=obs.time, y=t(data_mean[[2]]), basisobj=fbasis_list[[2]])
    k.est <- min(km.est)

    cca.r.est <- estimate.cca(d1,d2,  fbasis_list[[1]],  fbasis_list[[2]], fvalue_list[[1]], fvalue_list[[2]], ncan=k.est)
    print(cca.r.est$corr)

    par(bg="white", mfrow=c(2,2))
    #image.plot(cca.r.true$A1, col=viridis(64))
    image.plot(cca.r.est$A1,ylim=c(1,0), col=viridis(64))
    image.plot(pinv(Apinv_list[[1]]), ylim=c(1,0), col=viridis(64))

    #image.plot(cca.r.true$A2, col=viridis(64))
    image.plot(cca.r.est$A2, ylim=c(1,0), col=viridis(64))
    image.plot(pinv(Apinv_list[[2]]), ylim=c(1,0), col=viridis(64))
}

In [ ]:
lapply(N.level,run_simulation)

In [ ]:
fbasis_list <- list()
fvalue_list <- list()

for(m in 1:M){
    print(paste("Esimtating funcational basis of data modality",m))
    #adjust number of basis to estimate
    temp_values <- estimate.fpca.basis(c(0,1), obs.time, t(data_mean[[m]]), km.est[m], bs.nbasis=km.est[m]*3)$values
    #print(paste("Number of km before adjustment", km.est[m]))
    km.est[m]<-length(which(temp_values>1e-8))
    #print(paste("Number of km after adjustment", km.est[m]))

    fbasis_list[[m]] <- estimate.fpca.basis(c(0,1), obs.time, t(data_mean[[m]]), km.est[m], bs.nbasis=km.est[m]*3)$vectors

    fvalue_list[[m]] <- estimate.fpca.basis(c(0,1), obs.time, t(data_mean[[m]]), km.est[m], bs.nbasis=km.est[m]*3)$values[1:km.est[m]]
    print("Estimated Eigenvalues:")
    print(fvalue_list[[m]])
    print("True Eigenvalues:")
    print(true.values_list[[m]])
    
}

d1 <-Data2fd(argvals=obs.time, y=t(data_mean[[1]]), basisobj=fbasis_list[[1]])
#data modality 2

d2 <-Data2fd(argvals=obs.time, y=t(data_mean[[2]]), basisobj=fbasis_list[[2]])
k.est <- min(km.est)

cca.r.est <- estimate.cca(d1,d2,  fbasis_list[[1]],  fbasis_list[[2]], fvalue_list[[1]], fvalue_list[[2]], ncan=k.est)
#cca.r.true <- estimate.cca(d1,d2,  true.basis_list[[1]],  true.basis_list[[2]], true.values_list[[1]], true.values_list[[2]], ncan=k.est)

In [ ]:
print(cca.r.true$corr)
print(cca.r.est$corr)
A1.est <- cca.result$A1
A2.est <- cca.result$A2
par(bg="white", mfrow=c(2,2))
#image.plot(cca.r.true$A1, col=viridis(64))
image.plot(cca.r.est$A1, col=viridis(64))
image.plot(pinv(Apinv_list[[1]]), col=viridis(64))

#image.plot(cca.r.true$A2, col=viridis(64))
image.plot(cca.r.est$A2, col=viridis(64))
image.plot(pinv(Apinv_list[[2]]), col=viridis(64))

In [ ]:
dim(A1.est)
library(Matrix)

In [ ]:

print(cca.r.est$A1)
print(pinv(Apinv_list[[1]]))

In [ ]:
cca.r.true$A2 %*% t(pinv(Apinv_list[[2]]))

In [ ]:
ccafd <- varmx.cca.fd(cca.fd(d1, d2, ncan =5))
inprod(ccafd$weight1,fbasis_list[[1]])

In [ ]:
##
## Example with 2 different bases 
##
daybasis3 <- create.fourier.basis(c(0, 365))
daybasis5 <- create.fourier.basis(c(0, 365), 5)
tempfd3 <- with(CanadianWeather, smooth.basis(day.5, 
       dailyAv[,,"Temperature.C"], 
       daybasis3, fdnames=list("Day", "Station", "Deg C"))$fd )
precfd5 <- with(CanadianWeather, smooth.basis(day.5, 
       dailyAv[,,"log10precip"],
       daybasis5, fdnames=list("Day", "Station", "Deg C"))$fd )

# Compare with structure described above under 'value':
str(tempPrecVar3.5 <- var.fd(tempfd3, precfd5))

##
## The following produces contour and perspective plots
##

# Evaluate at a 53 by 53 grid for plotting

daybasis65 <- create.fourier.basis(rangeval=c(0, 365), nbasis=65)

daytempfd <- with(CanadianWeather, smooth.basis(day.5, 
       dailyAv[,,"Temperature.C"],
       daybasis65, fdnames=list("Day", "Station", "Deg C"))$fd )
str(tempvarbifd <- var.fd(daytempfd))

str(tempvarmat  <- eval.bifd(weeks,weeks,tempvarbifd))
# dim(tempvarmat)= c(53, 53)

op <- par(bg="white", mfrow=c(1,2), pty="s")
#contour(tempvarmat, xlab="Days", ylab="Days")
contour(weeks, weeks, tempvarmat, 
        xlab="Daily Average Temperature",
        ylab="Daily Average Temperature",
        main=paste("Variance function across locations\n",
          "for Canadian Anual Temperature Cycle"),
        cex.main=0.8, axes=FALSE)
axisIntervals(1, atTick1=seq(0, 365, length=5), atTick2=NA, 
            atLabels=seq(1/8, 1, 1/4)*365,
            labels=paste("Q", 1:4) )
axisIntervals(2, atTick1=seq(0, 365, length=5), atTick2=NA, 
            atLabels=seq(1/8, 1, 1/4)*365,
            labels=paste("Q", 1:4) )
persp(weeks, weeks, tempvarmat,
      xlab="Days", ylab="Days", zlab="Covariance")
mtext("Temperature Covariance", line=-4, outer=TRUE)
par(op)

In [ ]:
tempvarbifd

In [ ]:
estimate.fpca.sum.basis(c(0,1), obs.time, data[[1]], bs.nbasis=10)